In [81]:
import csv
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm
import numpy as np
import warnings

warnings.filterwarnings('ignore')


In [82]:
# Regression Results with Donation Change as Dummy 

a = "~/Documents/GitHub/CampaignDonations/Disney_Donor_Vote.csv"
df = pd.read_csv(a)


In [83]:
df = df.drop(columns = ["person", "name_x" ,"party_x", "party_y", "Last_name" , "Suffix" , "LastName", "ID_NameParty", "Pure_name" , "FirstName" , "first" , "last", "religion" ,"death" , "birthplace", "deathplace"])
df.head()

VoteAlong113thRatio  VoteAlong114thRatio  VoteAlong115thRatio  \
0                  NaN                  NaN                  NaN   
1                 0.00             0.666667                  NaN   
2                 0.75             0.333333                  0.6   
3                 0.50                  NaN                  NaN   
4                 0.00             0.333333                  0.4   

   VoteAlong116thRatio party_short  donations2012  donations2014  \
0                  NaN           N            NaN            NaN   
1                  NaN           D            NaN            NaN   
2                  1.0           D            NaN            NaN   
3                  NaN           D            NaN            NaN   
4                  1.0           D            0.0            0.0   

   donations2016  donations2018    pageid  ...        wikititle  \
0            NaN            NaN       NaN  ...              NaN   
1            NaN            NaN       NaN  ...              NaN   
2            NaN            NaN       NaN  ...              NaN   
3            NaN            NaN       NaN  ...              NaN   
4            0.0         1000.0  699496.0  ...  Earl_Blumenauer   

            name_y   sex ethnicity       birth Number_Sessions_Served  \
0              NaN   NaN       NaN         NaN                    NaN   
1              NaN   NaN       NaN         NaN                    NaN   
2              NaN   NaN       NaN         NaN                    NaN   
3              NaN   NaN       NaN         NaN                    NaN   
4  Earl Blumenauer  male     white  1948-08-16                    4.0   

   Session_113 Session_114 Session_115 Session_116  
0          NaN         NaN         NaN         NaN  
1          NaN         NaN         NaN         NaN  
2          NaN         NaN         NaN         NaN  
3          NaN         NaN         NaN         NaN  
4         True        True        True        True  

[5 rows x 21 columns]

In [77]:
# First Regression: Impact of VoteAlong113thRatio on change in donations between 113th and 114th Session
df_First = df[["VoteAlong113thRatio", "party_short", "donations2012", "donations2014", "Session_113" , "Session_114" ]]
df_First["donations2014"].describe()

# Making sure every MoH served both sessions
df_First = df_First.dropna(subset=['Session_113'])
df_First = df_First.dropna(subset=['Session_114'])

# Subsets NAs to 0
df_First= df_First.fillna(0)
        

for x in range(len(df_First)):
    if df_First["donations2014"].iloc[x] >= 1:  
        df_First["donations2014"].iloc[x] = 1 
    
for x in range(len(df_First)):
    if df_First["donations2012"].iloc[x] >= 1:  
        df_First["donations2012"].iloc[x] = 1 
    
df_First["Donation_Change"] = 1

for x in range(len(df_First)):
    if df_First["donations2012"].iloc[x] > df_First["donations2014"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 0 
    if df_First["donations2012"].iloc[x] < df_First["donations2014"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 2
        

    
df_First_D = df_First[df_First.party_short == "D"]

df_First_R = df_First[df_First.party_short == "R"]

df_First.head()

VoteAlong113thRatio party_short  donations2012  donations2014  \
4                  0.00           D            0.0            0.0   
10                 0.50           D            0.0            1.0   
12                 0.25           D            0.0            0.0   
14                 0.25           D            0.0            0.0   
15                 0.00           D            0.0            0.0   

    Session_113  Session_114  Donation_Change  
4          True         True                1  
10         True         True                2  
12         True         True                1  
14         True         True                1  
15         True         True                1

In [79]:
# Running the actual OLS model

# For Democrats 



X = df_First_D[["VoteAlong113thRatio"]]
Y = df_First_D[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     4.821
Date:                Sat, 27 Mar 2021   Prob (F-statistic):             0.0301
Time:                        18:34:08   Log-Likelihood:                -75.798
No. Observations:                 120   AIC:                             155.6
Df Residuals:                     118   BIC:                             161.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.9433    

In [80]:
# For Republicans

X = df_First_R[["VoteAlong113thRatio"]]
Y = df_First_R[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.1016
Date:                Sat, 27 Mar 2021   Prob (F-statistic):              0.751
Time:                        18:35:22   Log-Likelihood:                -43.097
No. Observations:                 104   AIC:                             90.19
Df Residuals:                     102   BIC:                             95.48
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.0540    

In [84]:
# Second Regression: Impact of VoteAlong114thRatio on change in donations between 114th and 115th Session
df_First = df[["VoteAlong114thRatio", "party_short", "donations2014", "donations2016", "Session_114" , "Session_115" ]]
df_First["donations2014"].describe()

# Making sure every MoH served both sessions
df_First = df_First.dropna(subset=['Session_114'])
df_First = df_First.dropna(subset=['Session_115'])

# Subsets NAs to 0
df_First= df_First.fillna(0)
        

for x in range(len(df_First)):
    if df_First["donations2016"].iloc[x] >= 1:  
        df_First["donations2016"].iloc[x] = 1 
    
for x in range(len(df_First)):
    if df_First["donations2014"].iloc[x] >= 1:  
        df_First["donations2014"].iloc[x] = 1 
    
df_First["Donation_Change"] = 1

for x in range(len(df_First)):
    if df_First["donations2014"].iloc[x] > df_First["donations2016"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 0 
    if df_First["donations2014"].iloc[x] < df_First["donations2016"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 2
        

    
df_First_D = df_First[df_First.party_short == "D"]

df_First_R = df_First[df_First.party_short == "R"]

df_First.head()





VoteAlong114thRatio party_short  donations2014  donations2016  \
4              0.333333           D            0.0            0.0   
10             0.666667           D            1.0            1.0   
12             1.000000           D            0.0            0.0   
14             0.333333           D            0.0            0.0   
15             0.000000           D            0.0            0.0   

    Session_114  Session_115  Donation_Change  
4          True         True                1  
10         True         True                1  
12         True         True                1  
14         True         True                1  
15         True         True                1

In [85]:
# Running the actual OLS model

# For Democrats 

X = df_First_D[["VoteAlong114thRatio"]]
Y = df_First_D[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.6832
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.410
Time:                        12:21:47   Log-Likelihood:                -58.682
No. Observations:                 120   AIC:                             121.4
Df Residuals:                     118   BIC:                             126.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.9623    

In [86]:
# For Republicans

X = df_First_R[["VoteAlong114thRatio"]]
Y = df_First_R[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.915
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.169
Time:                        12:21:50   Log-Likelihood:                -42.325
No. Observations:                 104   AIC:                             88.65
Df Residuals:                     102   BIC:                             93.94
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.8429    

In [91]:

# Third Regression: Impact of VoteAlong115thRatio on change in donations between 115th and 116th Session
df_First = df[["VoteAlong115thRatio", "party_short", "donations2016", "donations2018", "Session_115" , "Session_116" ]]

# Making sure every MoH served both sessions
df_First = df_First.dropna(subset=['Session_115'])
df_First = df_First.dropna(subset=['Session_116'])

# Subsets NAs to 0
df_First= df_First.fillna(0)
        

for x in range(len(df_First)):
    if df_First["donations2018"].iloc[x] >= 1:  
        df_First["donations2018"].iloc[x] = 1 
    
for x in range(len(df_First)):
    if df_First["donations2016"].iloc[x] >= 1:  
        df_First["donations2016"].iloc[x] = 1 
    
df_First["Donation_Change"] = 1

for x in range(len(df_First)):
    if df_First["donations2016"].iloc[x] > df_First["donations2018"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 0 
    if df_First["donations2016"].iloc[x] < df_First["donations2018"].iloc[x]:  
        df_First["Donation_Change"].iloc[x] = 2
        

    
df_First_D = df_First[df_First.party_short == "D"]

df_First_R = df_First[df_First.party_short == "R"]

df_First.head()



VoteAlong115thRatio party_short  donations2016  donations2018  \
4                   0.4           D            0.0            1.0   
10                  0.6           D            1.0            1.0   
12                  0.8           D            0.0            0.0   
14                  0.4           D            0.0            1.0   
15                  0.2           D            0.0            0.0   

    Session_115  Session_116  Donation_Change  
4          True         True                2  
10         True         True                1  
12         True         True                1  
14         True         True                2  
15         True         True                1

In [92]:
# Running the actual OLS model

# For Democrats 

X = df_First_D[["VoteAlong115thRatio"]]
Y = df_First_D[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9271
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.338
Time:                        12:26:57   Log-Likelihood:                -67.925
No. Observations:                 120   AIC:                             139.9
Df Residuals:                     118   BIC:                             145.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.1708    

In [94]:
# For Republicans

X = df_First_R[["VoteAlong115thRatio"]]
Y = df_First_R[["Donation_Change"]]

X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:        Donation_Change   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7800
Date:                Sun, 28 Mar 2021   Prob (F-statistic):              0.379
Time:                        12:28:39   Log-Likelihood:                -34.713
No. Observations:                 104   AIC:                             73.43
Df Residuals:                     102   BIC:                             78.72
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.0603    

In [ ]:
# Ideenspeicher

# Change Party-column to dummy
for x in range(len(df_First)):
    if df_First["party_short"].iloc[x] == "D":  
        df_First["party_short"].iloc[x] = 1 
        
for x in range(len(df_First)):
    if df_First["party_short"].iloc[x] == "R":  
        df_First["party_short"].iloc[x] = 0        


# Separate Data set into party-specific datasets

for x in range(len(df_First)):
    if df_First["party_short"].iloc[x] == "R":  
        df_First["party_short"].iloc[x] = 0    